<a href="https://colab.research.google.com/github/Tiblonde/pytorch-learning/blob/main/YOLOv3Implement.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

1.下载https://github.com/pjreddie/darknet/tree/master/cfg中的YOLOv3.cfg文件

In [1]:
# 把cfg文件放入cfg文件夹中
!mkdir cfg
%cd cfg
!wget https://github.com/pjreddie/darknet/raw/refs/heads/master/cfg/yolov3.cfg

/content/cfg
--2025-03-10 15:52:56--  https://github.com/pjreddie/darknet/raw/refs/heads/master/cfg/yolov3.cfg
Resolving github.com (github.com)... 140.82.112.4
Connecting to github.com (github.com)|140.82.112.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/pjreddie/darknet/refs/heads/master/cfg/yolov3.cfg [following]
--2025-03-10 15:52:56--  https://raw.githubusercontent.com/pjreddie/darknet/refs/heads/master/cfg/yolov3.cfg
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.109.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 8342 (8.1K) [text/plain]
Saving to: ‘yolov3.cfg’

yolov3.cfg          100%[===================>]   8.15K  --.-KB/s    in 0s      

2025-03-10 15:52:56 (25.2 MB/s) - ‘yolov3.cfg’ saved [8342/8342]



2.解析配置文件（这部分将创建一个darknet.py）

In [2]:
from __future__ import division

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
import numpy as np

In [4]:
def parse_cfg(cfgfile):

  """
  接受一个cfg文件

  返回一个模块列表。每个块描述了神经网络中要构建的模块。
  块以键值对列表形式储存。
  """
  file = open(cfgfile, "r")
  lines = file.read().split('\n') # 形成一个list
  lines = [x for x in lines if len(x) > 0] # 剔除空行
  lines = [x for x in lines if x[0] != '#'] # 剔除注释
  lines = [x.rstrip().lstrip() for x in lines]

  block = {}
  blocks = []

  for line in lines:
    if line[0] == "[":
      if len(block) != 0:
        blocks.append(block)
        block = {}
      block["type"] = line[1:-1].rstrip()
    else:
      key, value = line.split("=")
      block[key.rstrip()] = value.lstrip()
  blocks.append(block)

  return blocks



In [14]:
# parse_cfg函数测试
cfgfile = 'yolov3.cfg'
blocks = parse_cfg(cfgfile)

print(len(blocks))
blocks

108


[{'type': 'net',
  'batch': '64',
  'subdivisions': '16',
  'width': '608',
  'height': '608',
  'channels': '3',
  'momentum': '0.9',
  'decay': '0.0005',
  'angle': '0',
  'saturation': '1.5',
  'exposure': '1.5',
  'hue': '.1',
  'learning_rate': '0.001',
  'burn_in': '1000',
  'max_batches': '500200',
  'policy': 'steps',
  'steps': '400000,450000',
  'scales': '.1,.1'},
 {'type': 'convolutional',
  'batch_normalize': '1',
  'filters': '32',
  'size': '3',
  'stride': '1',
  'pad': '1',
  'activation': 'leaky'},
 {'type': 'convolutional',
  'batch_normalize': '1',
  'filters': '64',
  'size': '3',
  'stride': '2',
  'pad': '1',
  'activation': 'leaky'},
 {'type': 'convolutional',
  'batch_normalize': '1',
  'filters': '32',
  'size': '1',
  'stride': '1',
  'pad': '1',
  'activation': 'leaky'},
 {'type': 'convolutional',
  'batch_normalize': '1',
  'filters': '64',
  'size': '3',
  'stride': '1',
  'pad': '1',
  'activation': 'leaky'},
 {'type': 'shortcut', 'from': '-3', 'activatio

3.构建模块

In [ ]:
def create_modules(blocks):
  """
  接受一个键值对列表

  返回
  """
  net_info = blocks[0]
  module_list = nn.ModulesList()
  prev_filters = 3
  output_filters = []